In [20]:
import arcpy
import os

In [87]:
#Set workspace, initiate empty list that will house our Feature Classes
workspace = "N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\WyomingRoads2015.gdb"
fcs = []

#Walk the file geodatabase looking for the 100k quad Feature Classes we are interested in
walk = arcpy.da.Walk(workspace, datatype="FeatureClass")

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        if filename.endswith('QuadBoundaries'):
            fcs.append(os.path.join(dirpath, filename))
            
fcs

[u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\NorthWestBlock\\NW_100kQuadBoundaries',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\WestCentralBlock\\WC_100kQuadBoundaries',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\SouthWestBlock\\SW_100kQuadBoundaries',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\NorthEastBlock\\NE_100kQuadBoundaries',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\SouthEastBlock\\SE_100kQuadBoundaries']

In [60]:
#Dissolve our Feature Classes so that they consist of just one feature, export to shapefile.  
#Add shapefiles to an empty list.
diss_shps = []
for fc in fcs:
    out_feature = os.path.join("N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Dissovled_100kQuads", 
                               fc.rsplit('\\', 1)[-1] + '.shp')
    diss_shps.append(out_feature)
    arcpy.Dissolve_management(fc, out_feature)
    
diss_shps

[u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\QAQC\\Dissovled_100kQuads\\NW_100kQuadBoundaries.shp',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\QAQC\\Dissovled_100kQuads\\WC_100kQuadBoundaries.shp',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\QAQC\\Dissovled_100kQuads\\SW_100kQuadBoundaries.shp',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\QAQC\\Dissovled_100kQuads\\NE_100kQuadBoundaries.shp',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\QAQC\\Dissovled_100kQuads\\SE_100kQuadBoundaries.shp']

#### Select the randomly selected quads that have their centroid within each of the dissolved shapefiles.  Export the selection to new shapefiles.


In [82]:
# Set workspace, overwrite option, and variables.
arcpy.env.workspace = "N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Dissovled_100kQuads"
arcpy.env.overwriteOutput = True 
    
sel_quads = 'N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Selected24kQuads.shp'
clip_shps = []

for shp in diss_shps:
    out_clip = os.path.join("N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_24kQuads", 
                            shp.rsplit('\\', 1)[-1].split('_')[0] + '_24kQuads.shp')
    clip_shps.append(out_clip)
    
    arcpy.MakeFeatureLayer_management('N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Selected24kQuads.shp',
                                      "sel_quads")
    
    selection = arcpy.SelectLayerByLocation_management("sel_quads", "HAVE_THEIR_CENTER_IN", shp, "", "NEW_SELECTION")
    
    #Ensure the correct amount of features are being selected.
    matchcount = int(arcpy.GetCount_management(selection)[0]) 
    print(matchcount)
    
    arcpy.CopyFeatures_management(selection, out_clip) 

40
60
60
60
60


In [99]:
#Walk the file geodatabase looking for the road Feature Classes we are interested in
walk_rd = arcpy.da.Walk(workspace, datatype="FeatureClass")
fc_rd = []

for dirpath, dirnames, filenames in walk_rd:
    for filename in filenames:
        if 'Road' in filename:
            fc_rd.append(os.path.join(dirpath, filename))
            
roads = fc_rd[-5:]
roads

[u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\NorthWestBlock\\NW_WYRoads_2015_Clip',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\WestCentralBlock\\WC_Roads',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\SouthWestBlock\\SW_WYRoads_2015_Clip_1',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\NorthEastBlock\\NE_WYRoads_2015_Clip',
 u'N:\\GISData\\Active\\Bowen_WYRoads_2015\\DerivedData\\WyomingRoads2015.gdb\\SouthEastBlock\\SE_WYRoads_2015_Clip']

In [138]:
#Clip the roads layers to the randomly selected 24k quads, then create a point shapefile that contains randomly generated
#points, one for each road.

rnd_pt_list = []

for rd in roads:
    out_clip_rd = os.path.join("N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads", 
                               rd.rsplit('\\', 1)[-1].split('_')[0] + '_WYRoads_Clip.shp')
    arcpy.Clip_analysis(rd, sel_quads, out_clip_rd)
    arcpy.AddField_management(out_clip_rd, "Random", "SHORT")
    arcpy.CalculateField_management(out_clip_rd, "Random", "1", "PYTHON_9.3")
    rnd_pts = arcpy.CreateRandomPoints_management(out_path="N:/GISData/Active/Bowen_WYRoads_2015/DerivedData/QAQC/Random_Points", 
                                    out_name=(out_clip_rd.rsplit('\\', 1)[-1].rsplit('_', 1)[0] + '_RandomPt'),
                                    constraining_feature_class=out_clip_rd, constraining_extent="0 0 250 250",
                                    number_of_points_or_field="Random", minimum_allowed_distance="50 Meters",
                                    create_multipoint_output="POINT", multipoint_size="0")
    rnd_pt_list.append(rnd_pts)
    
    print(out_clip_rd)

N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads\NW_WYRoads_Clip.shp
N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads\WC_WYRoads_Clip.shp
N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads\SW_WYRoads_Clip.shp
N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads\NE_WYRoads_Clip.shp
N:\GISData\Active\Bowen_WYRoads_2015\DerivedData\QAQC\Clipped_Roads\SE_WYRoads_Clip.shp


In [161]:
#Randomly select half of the points to be assessed during QAQC.  This creates a more manageable, but still statistically
#significant subset.  Add fields that will be used to track errors and progress.

#Code block for input to Calculate Field operation.
codeblock = """
def rand_num():  
    import random  
    return random.randint(0,1)"""

for pt in rnd_pt_list:
    arcpy.AddField_management(pt, "Random", "SHORT")
    arcpy.AddField_management(pt, "Attr_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Pos_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Om_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Com_Err", "TEXT", field_length='1')
    arcpy.AddField_management(pt, "Assessed", "TEXT", field_length='1')
    arcpy.CalculateField_management(in_table=pt, field="Random", expression='rand_num()', expression_type="PYTHON_9.3", code_block=codeblock)